In [2]:
'''
Based on this article:  https://www.alpharithms.com/calculating-support-resistance-in-python-using-k-means-clustering-101517/
'''

import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import plotly.graph_objects as go
import ast
import plotly.express as px



import warnings
warnings.filterwarnings("ignore")


def get_data():
    # Load local CSV file and parse into dataframe object
    df = pd.read_csv('./data.csv').astype(float)
    for i in range(1,8):
        df[i]=float(0)
    return df.to_numpy()

def kmeans_analysis(data):
    cluster_arr = np.array([])
    for k in range(5,data.shape[0]):#data.shape[0]
        # Convert adjusted closing price to numpy array
        data_prices = data[:k+1,1]
        # print("data Prices:\n", data_prices)
        # Perform cluster analysis
        kmeans = KMeans(n_clusters=6).fit(data_prices.reshape(-1, 1))
        # predict which cluster each price is in
        clusters = kmeans.predict(data_prices.reshape(-1, 1))
        cluster_arr=np.append(cluster_arr, str(clusters.tolist()))
    data=pd.DataFrame(data)
    data.loc[5:5+len(cluster_arr)-1,'clusters']=cluster_arr


    df=data

    for start_row in range(5,data.shape[0]):#df.shape[0]
        data_prices=df.iloc[:start_row+1,1]
        clusters= ast.literal_eval(df.iloc[start_row,9]) #,  ,  , df.iloc[start_row,9]

        #min max calculation (changed the script for faster execution)
        min_max_values = np.zeros((6,2))
        min_max_values_arr=np.vstack((data_prices,clusters)).T
        for b in range(6):
            min_max_values[b, 1]=np.max(min_max_values_arr[min_max_values_arr[:,1]==b][:,0])
            min_max_values[b, 0]=np.min(min_max_values_arr[min_max_values_arr[:,1]==b][:,0])

        # Create container for combined values
        output = []
        # Sort based on cluster minimum
        s = sorted(min_max_values, key=lambda x: x[0])
        # For each cluster get average of
        for j, (_min, _max) in enumerate(s):
            # Append min from first cluster
            if j == 0:
                output.append(_min)
            # Append max from last cluster
            if j == len(min_max_values) - 1:
                output.append(_max)
            # Append average from cluster and adjacent for all others
            else:
                output.append(sum([_max, s[j + 1][0]]) / 2)
        # add the output columns to the df for latter plotting
        output = np.array(output)
        df.iloc[start_row, 2:9] = output



    for i in range(2,9):
        df['RM'+ str(i)]=df.iloc[:,i].rolling(100).mean() #df.iloc[:,i].rolling(100).mean(), df.iloc[:,i].rolling(5).mean()

    data=df

    # Assign an arbitrary color to each of the 6 clusters
    colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

    # Create Scatter plot, assigning each point a color where
    data['color']=[colors[i] for i in clusters]
    data['clusters']=clusters
    data.rename(columns = {1:'y'}, inplace = True)
    data['x']=data.index.tolist()
    data['clusters'] = data['clusters'].astype(str)


    # fig = data.plot.scatter(
    fig = px.scatter(data,
        x="x",
        y="y",#data.y ,pd.DataFrame({'y':data[1]}).all()
        title="Clustered Pricing K = 6",
        color="clusters",
    )

    # Add a trace of the price for better clarity
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['y'],
        line_color="black",
        line_width=1
    ))
    # Make it pretty
    layout = go.Layout(
        plot_bgcolor='#efefef',
        showlegend=False,
        # Font Families
        font_family='Monospace',
        font_color='#000000',
        font_size=20,
        xaxis=dict(
            rangeslider=dict(
                visible=False
            ))
    )
    fig.update_layout(layout)

    # Add rolling averages
    for col,cluster_avg in df.loc[:,'RM3':'RM7'].iteritems():#output df['RM2':'RM9']
        fig.add_scatter(x=df.index.tolist()[104:], y=cluster_avg[104:])


    fig.show()


if __name__ == "__main__":
    df = get_data()
    kmeans_analysis(df)
